# Boroughs of Toronto/Cluster Analysis

In [3]:
#downloading libraries
from bs4 import BeautifulSoup
import requests
import csv

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from pandas.io.json import json_normalize 
import seaborn as sns

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 


import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans # import k-means from clustering stage

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


# Part1-Q1-4

In [4]:
###web scarping from the Wikipedia site
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(URL).text
soup = BeautifulSoup(source,'html.parser')

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
##print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [6]:
df.head()

PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

In [7]:
print(df)

    PostalCode                 Borough  \
0          M3A              North York   
1          M4A              North York   
2          M5A        Downtown Toronto   
3          M6A              North York   
4          M7A            Queen's Park   
5          M9A               Etobicoke   
6          M1B             Scarborough   
7          M3B              North York   
8          M4B               East York   
9          M5B        Downtown Toronto   
10         M6B              North York   
11         M9B               Etobicoke   
12         M1C             Scarborough   
13         M3C              North York   
14         M4C               East York   
15         M5C        Downtown Toronto   
16         M6C                    York   
17         M9C               Etobicoke   
18         M1E             Scarborough   
19         M4E            East Toronto   
20         M5E        Downtown Toronto   
21         M6E                    York   
22         M1G             Scarbor

In [8]:
df.shape

(103, 3)

# Part2 - Latitude and Longitude

In [9]:
lat_long = pd.read_csv(r"C:\Users\cmumcu\Desktop\Geospatial_Coordinates.csv")
lat_long.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [10]:
lat_long.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_new = pd.merge(df,lat_long,on='PostalCode')
df_new.head()

PostalCode           Borough                      Neighborhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto         Regent Park, Harbourfront  43.654260   
3        M6A        North York  Lawrence Manor, Lawrence Heights  43.718518   
4        M7A      Queen's Park     Ontario Provincial Government  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494

# Part3 - Clustering Neighborhoods of Toronto

In [11]:
##Examining Downtown Toronto Borough
df_DowntownToronto = df_new[df_new['Borough'].str.contains('Downtown Toronto',regex=False)]
df_DowntownToronto

PostalCode                 Borough  \
2         M5A        Downtown Toronto   
9         M5B        Downtown Toronto   
15        M5C        Downtown Toronto   
20        M5E        Downtown Toronto   
24        M5G        Downtown Toronto   
25        M6G        Downtown Toronto   
30        M5H        Downtown Toronto   
36        M5J        Downtown Toronto   
42        M5K        Downtown Toronto   
48        M5L        Downtown Toronto   
80        M5S        Downtown Toronto   
84        M5T        Downtown Toronto   
87        M5V        Downtown Toronto   
91        M4W        Downtown Toronto   
92        M5W  Downtown Toronto Stn A   
96        M4X        Downtown Toronto   
97        M5X        Downtown Toronto   
99        M4Y        Downtown Toronto   

                                         Neighborhood   Latitude  Longitude  
2                           Regent Park, Harbourfront  43.654260 -79.360636  
9                            Garden District, Ryerson  43.657162 -79.378937  
15                                     St. James Town  43.651494 -79.375418  
20                                        Berczy Park  43.644771 -79.373306  
24                                 Central Bay Street  43.657952 -79.387383  
25                                           Christie  43.669542 -79.422564  
30                           Richmond, Adelaide, King  43.650571 -79.384568  
36  Harbourfront East, Union Station, Toronto Islands  43.640816 -79.381752  
42           Toronto Dominion Centre, Design Exchange  43.647177 -79.381576  
48                     Commerce Court, Victoria Hotel  43.648198 -79.379817  
80                     University of Toronto, Harbord  43.662696 -79.400049  
84          Kensington Market, Chinatown, Grange Park  43.653206 -79.400049  
87  CN Tower, King and Spadina, Railway Lands, Har...  43.628947 -79.394420  
91                                           Rosedale  43.679563 -79.377529  
92                                     Enclave of M5E  43.646435 -79.374846  
96                        St. James Town, Cabbagetown  43.667967 -79.367675  
97             First Canadian Place, Underground city  43.648429 -79.382280  
99                               Church and Wellesley  43.665860 -79.383160

In [12]:
df_DowntownToronto.shape

(18, 5)

In [15]:
#Using FourSquare API
CLIENT_ID = 'I5BNVOQGS2VPHWYQCIVWRS4NVW1E3FHMUPRYNL5BT5EX1ZJ1' #  Foursquare ID
CLIENT_SECRET = 'KFMEF1Q51W4VD0EJUY5DP0KLQA5EXRQ1BWEVOZQZYOISDVHE' #  Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    radius=500
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [17]:
DowntownToronto_venues = getNearbyVenues(names=df_DowntownToronto['Neighborhood'],
                                   latitudes=df_DowntownToronto['Latitude'],
                                   longitudes=df_DowntownToronto['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
Found 1190 venues in 18 neighborhoods.


In [18]:
print(DowntownToronto_venues.shape)
DowntownToronto_venues.head()

(1190, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2  Cooper Koo Family YMCA       43.653249       -79.358008   
3     Body Blitz Spa East       43.654735       -79.359874   
4          Impact Kitchen       43.656369       -79.356980   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2  Distribution Center  
3                  Spa  
4           Restaurant

In [19]:
DowntownToronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            60   
CN Tower, King and Spadina, Railway Lands, Harb...                     14   
Central Bay Street                                                     62   
Christie                                                               16   
Church and Wellesley                                                   77   
Commerce Court, Victoria Hotel                                        100   
Enclave of M5E                                                         98   
First Canadian Place, Underground city                                100   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
Kensington Market, Chinatown, Grange Park                              62   
Regent Park, Harbourfront                                              45   
Richmond, Adelaide, King                                               94   
Rosedale                                                                4   
St. James Town                                                         81   
St. James Town, Cabbagetown                                            45   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         32   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             60   
CN Tower, King and Spadina, Railway Lands, Harb...                      14   
Central Bay Street                                                      62   
Christie                                                                16   
Church and Wellesley                                                    77   
Commerce Court, Victoria Hotel                                         100   
Enclave of M5E                                                          98   
First Canadian Place, Underground city                                 100   
Garden District, Ryerson                                               100   
Harbourfront East, Union Station, Toronto Islands                      100   
Kensington Market, Chinatown, Grange Park                               62   
Regent Park, Harbourfront                                               45   
Richmond, Adelaide, King                                                94   
Rosedale                                                                 4   
St. James Town                                                          81   
St. James Town, Cabbagetown                                             45   
Toronto Dominion Centre, Design Exchange                               100   
University of Toronto, Harbord                                          32   

                                                    Venue  Venue Latitude  \
Neighborhood                                                                
Berczy Park                                            60              60   
CN Tower, King and Spadina, Railway Lands, Harb...     14              14   
Central Bay Street                                     62              62   
Christie                                               16              16   
Church and Wellesley                                   77              77   
Commerce Court, Victoria Hotel                        100             100   
Enclave of M5E                                         98              98   
First Canadian Place, Underground city                100             100   
Garden District, Ryerson                              100             100   
Harbourfront East, Union Station, Toronto Islands     100             100   
Kensington Market, Chinato

In [20]:
print('There are {} distinct venues in {} categories.'.format(
    len(DowntownToronto_venues['Venue'].unique()),len(DowntownToronto_venues['Venue Category'].unique())))

There are 701 distinct venues in 205 categories.


In [21]:
# one hot encoding
DowntownToronto_onehot = pd.get_dummies(DowntownToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DowntownToronto_onehot['Neighborhood'] = DowntownToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DowntownToronto_onehot.columns[-1]] + list(DowntownToronto_onehot.columns[:-1])
DowntownToronto_onehot = DowntownToronto_onehot[fixed_columns]

DowntownToronto_onehot.head()

Yoga Studio  Adult Boutique  Airport  Airport Food Court  Airport Gate  \
0            0               0        0                   0             0   
1            0               0        0                   0             0   
2            0               0        0                   0             0   
3            0               0        0                   0             0   
4            0               0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0         0            0           0                    0   
1             0         0            0           0                    0   
2             0         0            0           0                    0   
3             0         0            0           0                    0   
4             0         0            0           0                    0   

   Asian Restaurant  Athletics & Sports  BBQ Joint  Baby Store  Bagel Shop  \
0                 0                   0          0           0           0   
1                 0                   0          0           0           0   
2                 0                   0          0           0           0   
3                 0                   0          0           0           0   
4                 0                   0          0           0           0   

   Bakery  Bank  Bar  Baseball Stadium  Basketball Stadium  Beach  \
0       1     0    0                 0                   0      0   
1       0     0    0                 0                   0      0   
2       0     0    0                 0                   0      0   
3       0     0    0                 0                   0      0   
4       0     0    0                 0                   0      0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bistro  \
0                0         0           0                   0       0   
1                0         0           0                   0       0   
2                0         0           0                   0       0   
3                0         0           0                   0       0   
4                0         0           0                   0       0   

   Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  \
0              0          0         0                     0               0   
1              0          0         0                     0               0   
2              0          0         0                     0               0   
3              0          0         0                     0               0   
4              0          0         0                     0               0   

   Brewery  Bubble Tea Shop  Building  Burger Joint  Burrito Place  Butcher  \
0        0                0         0             0              0        0   
1        0                0         0             0              0        0   
2        0                0         0             0              0        0   
3        0                0         0             0              0        0   
4        0                0         0             0              0        0   

   Café  Candy Store  Caribbean Restaurant  Cheese Shop  Chinese Restaurant  \
0     0            0                     0            0                   0   
1     0            0                     0            0                   0   
2     0            0                     0            0                   0   
3     0            0                     0 

In [22]:
# Group by neighborhood via mean of category occurences 
DowntownToronto_grouped = DowntownToronto_onehot.groupby('Neighborhood').mean().reset_index()
DowntownToronto_grouped

Neighborhood  Yoga Studio  \
0                                         Berczy Park     0.000000   
1   CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
2                                  Central Bay Street     0.016129   
3                                            Christie     0.000000   
4                                Church and Wellesley     0.025974   
5                      Commerce Court, Victoria Hotel     0.000000   
6                                      Enclave of M5E     0.010204   
7              First Canadian Place, Underground city     0.000000   
8                            Garden District, Ryerson     0.000000   
9   Harbourfront East, Union Station, Toronto Islands     0.000000   
10          Kensington Market, Chinatown, Grange Park     0.000000   
11                          Regent Park, Harbourfront     0.022222   
12                           Richmond, Adelaide, King     0.000000   
13                                           Rosedale     0.000000   
14                                     St. James Town     0.000000   
15                        St. James Town, Cabbagetown     0.000000   
16           Toronto Dominion Centre, Design Exchange     0.000000   
17                     University of Toronto, Harbord     0.031250   

    Adult Boutique   Airport  Airport Food Court  Airport Gate  \
0         0.000000  0.000000            0.000000      0.000000   
1         0.000000  0.071429            0.071429      0.071429   
2         0.000000  0.000000            0.000000      0.000000   
3         0.000000  0.000000            0.000000      0.000000   
4         0.012987  0.000000            0.000000      0.000000   
5         0.000000  0.000000            0.000000      0.000000   
6         0.000000  0.000000            0.000000      0.000000   
7         0.000000  0.000000            0.000000      0.000000   
8         0.000000  0.000000            0.000000      0.000000   
9         0.000000  0.000000            0.000000      0.000000   
10        0.000000  0.000000            0.000000      0.000000   
11        0.000000  0.000000            0.000000      0.000000   
12        0.000000  0.000000            0.000000      0.000000   
13        0.000000  0.000000            0.000000      0.000000   
14        0.000000  0.000000            0.000000      0.000000   
15        0.000000  0.000000            0.000000      0.000000   
16        0.000000  0.000000            0.000000      0.000000   
17        0.000000  0.000000            0.000000      0.000000   

    Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0         0.000000         0.000000          0.000000             0.000000   
1         0.071429         0.142857          0.071429             0.000000   
2         0.000000         0.000000          0.000000             0.000000   
3         0.000000         0.000000          0.000000             0.000000   
4         0.000000         0.000000          0.000000             0.012987   
5         0.000000         0.000000          0.000000             0.030000   
6         0.000000         0.000000          0.000000             0.000000   
7         0.000000         0.000000          0.000000             0.030000   
8         0.000000         0.000000          0.000000             0.000000   
9         0.000000         0.000000          0.000000             0.000000   
10        0.000000         0.000000          0.000000             0.000000   
11        0.000000         0.000000          0.000000             0.000000   
12        0.000000         0.000000          0.000000             0.021277   
13        0.000000         0.000000          0.000000             0.000000   
14        0.000000         0.000000          0.000000             0.024691   
15        0.000000         0.000000          0.000000             0.000000   
16        0.000000         0.000000          0.000000             0.020000   
17        0.000000         0.000000          0.0000

In [42]:
#Top 10 most common venues at each neighborhood 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DowntownToronto_grouped['Neighborhood']

for ind in np.arange(DowntownToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DowntownToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

Neighborhood 1st Most Common Venue  \
0                                         Berczy Park           Coffee Shop   
1   CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
2                                  Central Bay Street           Coffee Shop   
3                                            Christie         Grocery Store   
4                                Church and Wellesley           Coffee Shop   
5                      Commerce Court, Victoria Hotel           Coffee Shop   
6                                      Enclave of M5E           Coffee Shop   
7              First Canadian Place, Underground city           Coffee Shop   
8                            Garden District, Ryerson           Coffee Shop   
9   Harbourfront East, Union Station, Toronto Islands           Coffee Shop   
10          Kensington Market, Chinatown, Grange Park                  Café   
11                          Regent Park, Harbourfront           Coffee Shop   
12                           Richmond, Adelaide, King           Coffee Shop   
13                                           Rosedale                  Park   
14                                     St. James Town           Coffee Shop   
15                        St. James Town, Cabbagetown           Pizza Place   
16           Toronto Dominion Centre, Design Exchange           Coffee Shop   
17                     University of Toronto, Harbord                  Café   

   2nd Most Common Venue          3rd Most Common Venue  \
0           Cocktail Bar                         Bakery   
1          Boat or Ferry                       Boutique   
2         Sandwich Place                           Café   
3                   Café                           Park   
4    Japanese Restaurant               Sushi Restaurant   
5             Restaurant                          Hotel   
6           Cocktail Bar             Seafood Restaurant   
7                   Café                          Hotel   
8         Clothing Store                 Cosmetics Shop   
9               Aquarium                          Hotel   
10    Mexican Restaurant  Vegetarian / Vegan Restaurant   
11                Bakery                           Park   
12                  Café                     Restaurant   
13                 Trail                     Playground   
14                  Café                 Cosmetics Shop   
15            Restaurant                    Coffee Shop   
16                 Hotel                           Café   
17                   Bar            Japanese Restaurant   

        4th Most Common Venue  5th Most Common Venue 6th Most Common Venue  \
0                 Cheese Shop             Restaurant    Seafood Restaurant   
1         Rental Car Location        Harbor / Marina      Sculpture Garden   
2          Italian Restaurant            Salad Place       Bubble Tea Shop   
3                 Candy Store            Coffee Shop             Nightclub   
4                  Restaurant                Gay Bar           Yoga Studio   
5                        Café     Italian Restaurant                   Gym   
6                      Bakery                   Café   Japanese Restaurant   
7         Japanese Restaurant             Restaurant                   Gym   
8   Middle Eastern Restaurant                   Café   Japanese Restaurant   
9                        Café     Italian Restaurant   Fried Chicken Joint   
10                Coffee Shop  Vietnamese Restaurant                   Bar   
11                       Café                    Pub        Breakfast Spot   
12             Clothing Store        Thai Restaurant                   Gym   
13               Cupcake Shop             Donut Shop      Doner Restaurant   
14               Cocktail Bar                  Hotel             Gastropub   
15                       Café                    Pub                Bakery   
16         Italian Restaurant            Salad Place    Seafood Restaurant   
17           

In [57]:
kclusters = 4

DowntownToronto_grouped_clustering = DowntownToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DowntownToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:17]

array([1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])

In [65]:
DowntownToronto_merged

PostalCode                 Borough  \
2         M5A        Downtown Toronto   
9         M5B        Downtown Toronto   
15        M5C        Downtown Toronto   
20        M5E        Downtown Toronto   
24        M5G        Downtown Toronto   
25        M6G        Downtown Toronto   
30        M5H        Downtown Toronto   
36        M5J        Downtown Toronto   
42        M5K        Downtown Toronto   
48        M5L        Downtown Toronto   
80        M5S        Downtown Toronto   
84        M5T        Downtown Toronto   
87        M5V        Downtown Toronto   
91        M4W        Downtown Toronto   
92        M5W  Downtown Toronto Stn A   
96        M4X        Downtown Toronto   
97        M5X        Downtown Toronto   
99        M4Y        Downtown Toronto   

                                         Neighborhood   Latitude  Longitude  \
2                           Regent Park, Harbourfront  43.654260 -79.360636   
9                            Garden District, Ryerson  43.657162 -79.378937   
15                                     St. James Town  43.651494 -79.375418   
20                                        Berczy Park  43.644771 -79.373306   
24                                 Central Bay Street  43.657952 -79.387383   
25                                           Christie  43.669542 -79.422564   
30                           Richmond, Adelaide, King  43.650571 -79.384568   
36  Harbourfront East, Union Station, Toronto Islands  43.640816 -79.381752   
42           Toronto Dominion Centre, Design Exchange  43.647177 -79.381576   
48                     Commerce Court, Victoria Hotel  43.648198 -79.379817   
80                     University of Toronto, Harbord  43.662696 -79.400049   
84          Kensington Market, Chinatown, Grange Park  43.653206 -79.400049   
87  CN Tower, King and Spadina, Railway Lands, Har...  43.628947 -79.394420   
91                                           Rosedale  43.679563 -79.377529   
92                                     Enclave of M5E  43.646435 -79.374846   
96                        St. James Town, Cabbagetown  43.667967 -79.367675   
97             First Canadian Place, Underground city  43.648429 -79.382280   
99                               Church and Wellesley  43.665860 -79.383160   

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
2                1           Coffee Shop                Bakery   
9                1           Coffee Shop        Clothing Store   
15               1           Coffee Shop                  Café   
20               1           Coffee Shop          Cocktail Bar   
24               1           Coffee Shop        Sandwich Place   
25               3         Grocery Store                  Café   
30               1           Coffee Shop                  Café   
36               1           Coffee Shop              Aquarium   
42               1           Coffee Shop                 Hotel   
48               1           Coffee Shop            Restaurant   
80               1                  Café                   Bar   
84               1                  Café    Mexican Restaurant   
87               2       Airport Service         Boat or Ferry   
91               0                  Park                 Trail   
92               1           Coffee Shop          Cocktail Bar   
96               1           Pizza Place            Restaurant   
97               1           Coffee Shop                  Café   
99               1           Coffee Shop   Japanese Restaurant   

            3rd Most Common Venue      4th Most Common Venue  \
2                            Park                       Café   
9                  Cosmetics Shop  Middle Eastern Restaurant   
15                 Cosmetics Shop               Cocktail Bar   
20                         Bakery                Cheese Shop   
24                           Café         Italian Restaurant   
25                           Park                Candy Store   
30                

In [66]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DowntownToronto_merged['Latitude'], DowntownToronto_merged['Longitude'], DowntownToronto_merged['Neighborhood'], DowntownToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [69]:
#Cluster 1
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 0, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
91  Downtown Toronto               0                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
91                 Trail            Playground          Cupcake Shop   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
91            Donut Shop      Doner Restaurant               Dog Run   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
91   Distribution Center        Discount Store                  Diner

In [70]:
#Cluster 2
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 1, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
2         Downtown Toronto               1           Coffee Shop   
9         Downtown Toronto               1           Coffee Shop   
15        Downtown Toronto               1           Coffee Shop   
20        Downtown Toronto               1           Coffee Shop   
24        Downtown Toronto               1           Coffee Shop   
30        Downtown Toronto               1           Coffee Shop   
36        Downtown Toronto               1           Coffee Shop   
42        Downtown Toronto               1           Coffee Shop   
48        Downtown Toronto               1           Coffee Shop   
80        Downtown Toronto               1                  Café   
84        Downtown Toronto               1                  Café   
92  Downtown Toronto Stn A               1           Coffee Shop   
96        Downtown Toronto               1           Pizza Place   
97        Downtown Toronto               1           Coffee Shop   
99        Downtown Toronto               1           Coffee Shop   

   2nd Most Common Venue          3rd Most Common Venue  \
2                 Bakery                           Park   
9         Clothing Store                 Cosmetics Shop   
15                  Café                 Cosmetics Shop   
20          Cocktail Bar                         Bakery   
24        Sandwich Place                           Café   
30                  Café                     Restaurant   
36              Aquarium                          Hotel   
42                 Hotel                           Café   
48            Restaurant                          Hotel   
80                   Bar            Japanese Restaurant   
84    Mexican Restaurant  Vegetarian / Vegan Restaurant   
92          Cocktail Bar             Seafood Restaurant   
96            Restaurant                    Coffee Shop   
97                  Café                          Hotel   
99   Japanese Restaurant               Sushi Restaurant   

        4th Most Common Venue  5th Most Common Venue 6th Most Common Venue  \
2                        Café                    Pub        Breakfast Spot   
9   Middle Eastern Restaurant                   Café   Japanese Restaurant   
15               Cocktail Bar                  Hotel             Gastropub   
20                Cheese Shop             Restaurant    Seafood Restaurant   
24         Italian Restaurant            Salad Place       Bubble Tea Shop   
30             Clothing Store        Thai Restaurant                   Gym   
36                       Café     Italian Restaurant   Fried Chicken Joint   
42         Italian Restaurant            Salad Place    Seafood Restaurant   
48                       Café     Italian Restaurant                   Gym   
80                  Bookstore                 Bakery           Yoga Studio   
84                Coffee Shop  Vietnamese Restaurant                   Bar   
92                     Bakery                   Café   Japanese Restaurant   
96                       Café                    Pub                Bakery   
97        Japanese Restaurant             Restaurant                   Gym   
99                 Restaurant                Gay Bar           Yoga Studio   

   7th Most Common Venue 8th Most Common Venue       9th Most Common Venue  \
2                Theater            Restaurant                        Bank   
9        Bubble Tea Shop               Theater                       Hotel   
15   American Restaurant    Italian Restaurant                        Park   
20              Pharmacy              Beer Bar              Farmers Market   
24          Burger Joint              Wine Bar  Modern European Restaurant   
30         Deli / Bodega      Sushi Restaurant               Burrito Place   
36            Restaurant        Scenic Lookout                     Brewery   
42   Japanese Restaurant            Restaurant              Breakfast Spot   
48   American Restaurant    Seafoo

In [71]:
#Cluster 3
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 2, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
87  Downtown Toronto               2       Airport Service   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
87         Boat or Ferry              Boutique   Rental Car Location   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
87       Harbor / Marina      Sculpture Garden      Airport Terminal   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
87           Coffee Shop        Airport Lounge           Airport Gate

In [72]:
#Cluster 4
DowntownToronto_merged.loc[DowntownToronto_merged['Cluster Labels'] == 3, DowntownToronto_merged.columns[[1] + list(range(5, DowntownToronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
25  Downtown Toronto               3         Grocery Store   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
25                  Café                  Park           Candy Store   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
25           Coffee Shop             Nightclub    Athletics & Sports   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
25    Italian Restaurant            Baby Store             Restaurant